<a href="https://colab.research.google.com/github/satyams00/Coverless-Text-Stenography/blob/main/Copy_of_Coverless_text_stenography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import pipeline
import random

In [ ]:
checkpoint = "MBZUAI/LaMini-Flan-T5-248M"

model = pipeline('text2text-generation', model = checkpoint)

text=["human body","science","earth","stars","animal","electronic acessories"]

generated_text =''

text = random.choice(text)
def get_cover_text():
    for i in range(0,10):
        global generated_text
        str = "Generate facts about "
        str += text + "in one sentence"
        generated_text += model(str, max_length=50, do_sample=True)[0]['generated_text']
    return generated_text


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
import jellyfish
import numpy as np

def jaro_winkler_similarity(str1, str2):
    return jellyfish.jaro_winkler_similarity(str1, str2)

def calculate_correlation(cover_text, stego_text):
    min_length = min(len(cover_text), len(stego_text))
    cover_values = [ord(c) for c in cover_text[:min_length]]
    stego_values = [ord(c) for c in stego_text[:min_length]]
    return np.corrcoef(cover_values, stego_values)[0, 1]

def embed_secret_message(cover_text, secret_message):
    cover_chars = list(cover_text)
    secret_chars = list(secret_message)

    unique_chars = []
    for char in reversed(cover_chars):
        if char not in unique_chars:
            unique_chars.append(char)

    secret_index = 0
    for char in unique_chars:
        for index, cover_char in enumerate(cover_chars):
            if cover_char == char and secret_index < len(secret_chars):
                cover_chars[index] = chr((ord(cover_chars[index]) ^ ord(secret_chars[secret_index])) % 256)
                secret_index += 1
        if secret_index >= len(secret_chars):
            break

    if secret_index < len(secret_chars):
        raise ValueError("Not enough characters to embed the secret message.")

    return ''.join(cover_chars).replace('\n', '')

# Function to extract the secret message from the stego text
def extract_secret_message(stego_text, cover_text, secret_length):
    stego_chars = list(stego_text)
    cover_chars = list(cover_text)

    # Start with the last character of the cover text
    unique_chars = []
    for char in reversed(cover_chars):
        if char not in unique_chars:
            unique_chars.append(char)

    secret_chars = []
    secret_index = 0
    for char in unique_chars:
        for index, cover_char in enumerate(cover_chars):
            if cover_char == char and secret_index < secret_length:
                secret_chars.append(chr((ord(stego_chars[index]) ^ ord(cover_chars[index])) % 256))
                secret_index += 1
        if secret_index >= secret_length:
            break

    return ''.join(secret_chars)

cover_text = get_cover_text()
print("Cover Text: ", cover_text)

msg = input("Enter secret message: ")
password = input("Create key: ")

d_sc = {}
d_cc = {}


try:
    stegotext = embed_secret_message(cover_text, msg)
    cover_steg_key = input("Create stego key: ")
    d_sc[cover_steg_key] = stegotext
    d_cc[stegotext] = msg

    jaro_winkler = jaro_winkler_similarity(cover_text, stegotext)
    print(f"Jaro-Winkler Similarity: {jaro_winkler:.4f}")

    correlation = calculate_correlation(cover_text, stegotext)
    print(f"Correlation: {correlation:.4e}")

    print("Stego Object: ", stegotext)
except ValueError as e:
    print(e)
    exit()

reveal = input("Do you want to reveal the message? (y/n) ")
if reveal == 'y':
    def steg_ci():
        count = 0
        while count < 5:
            cover_r = input("Enter Stego Object: ")
            skey = input("Enter stego key: ")
            if skey in d_sc.keys() and cover_r == d_sc[skey]:
                return d_sc[skey]
            else:
                print("Incorrect stego object or key")
                count += 1
        else:
            print("Exceeded limit")
            exit()

    def cipher_sec(stego):
        secret_length = len(msg)
        extracted_msg = extract_secret_message(stego, cover_text, secret_length)
        print("Secret message:", extracted_msg)

    stego = steg_ci()
    key = input("Enter Key: ")
    if key == password:
        cipher_sec(stego)
    else:
        print("Incorrect key")
        exit()


Cover Text:  The human body contains approximately 76% water, with 10% carbon dioxide, around 95% water and 20% oxygen.The human body consists of the muscles, bones, digestive system, blood vessels, and organs.The human body consists of 206 organs and 72 bones.Human body contains all living organisms that breathe, carry blood, exchange oxygen and carbon dioxide through blood and urine, and maintains balance of physical and chemical functions.The human body is composed of multiple body types, including the nervous system, heart, lungs, skin, and bones.Human body contains a large number of organs that work together to function.The human body weighs about 86 pounds and is primarily composed of bones.The human body is composed of organs and tissues that work together to function as humans do.The human body consists of various organs and tissues.The human body is a complex organ filled with hundreds of thousands of organs and tissues.The human body is a complex network of organs, tissues, a